In [ ]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

#Helpers
import numpy as np
import random
import os

2023-11-10 18:59:17.820322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/psabnac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 2. Types of Spaces

In [12]:
Discrete(3)
#Discrete(3).sample()

0

In [13]:
Box(0,1,shape=(3,3)).sample() #lower_value, upper value, shape

array([[0.942542  , 0.09328385, 0.25131196],
       [0.7601594 , 0.7532804 , 0.6432958 ],
       [0.07376474, 0.29331708, 0.09366215]], dtype=float32)

In [14]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 85, 178, 154],
       [107, 205,  75],
       [202, 105, 231]])

In [ ]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

In [20]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([50.972355], dtype=float32))])

In [28]:
MultiBinary(4).sample()

array([1, 0, 1, 0], dtype=int8)

In [36]:
MultiDiscrete([5,2,2]).sample()

array([0, 1, 1])

# 3. Building an Environment

In [ ]:
# Build an agent to give us the best shower possible
# Randomly temperature
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        #self.observation_space = Box(low=0, high=100, shape=(1,)) #Alternative
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60

    def step(self, action):
        # Apply action: Discrete 3 because notice
        # 0 -1 = -1 temperature
        # 1 -1 = 0  temperature remains the same
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [ ]:
env=ShowerEnv()
env.observation_space.sample()

In [ ]:
env.reset()

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(env, warn=True)

# 4. Test Environment

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [ ]:
env.close()

# 5. Train Model

In [ ]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=400000)

# 6. Save Model

In [ ]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')
model.save(shower_path)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
del model

In [ ]:
model = PPO.load(shower_path)

In [ ]:
evaluate_policy(model, env, n_eval_episodes =10, render=False)

Hyperparameter Tuning

Building custom environment details(PyGame)

Integration into other simulation systems (Mujoco, Unity)

End-to-end implementation: RL for Trading or Simulated to Real